# Setup Amazon EventBridge To Trigger a Pipeline Execution with S3

Amazon EventBridge is a serverless event bus that makes it easy to connect applications together using data from your own applications, integrated Software-as-a-Service (SaaS) applications, and AWS services.

You can choose an event source (i.e. Amazon S3) and select a target from a number of AWS services including AWS Step Functions, AWS Lambda, Amazon SNS, and Amazon Kinesis Data Firehose. Amazon EventBridge will automatically deliver the events in near real-time.

<img src="img/automated_pipeline.png" width="90%" align="left">

In [ ]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
import json
from botocore.exceptions import ClientError

sess   = sagemaker.Session()
bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
account_id = boto3.client('sts').get_caller_identity().get('Account')

## Steps
1. Create S3 Buckets
2. Enable CloudTrail Logging
3. Get StepFunctions Pipeline
4. Create EventBridge Rule
5. Test Trigger

# Create S3 Data Upload Bucket (watched) & S3 Bucket for CloudTrail Logs

In [ ]:
watched_bucket = 'dsoaws-test-upload-{}'.format(account_id)
print(watched_bucket)

In [ ]:
!aws s3 mb s3://$watched_bucket

In [ ]:
!aws s3 ls $watched_bucket

In [ ]:
cloudtrail_bucket = 'cloudtrail-dsoaws-{}'.format(account_id)
print(cloudtrail_bucket)

In [ ]:
!aws s3 mb s3://$cloudtrail_bucket

In [ ]:
!aws s3 ls $cloudtrail_bucket

# Attach an S3 Policy to the Cloud Trail ^^ Logging Bucket ^^ Above

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AWSCloudTrailAclCheck20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:GetBucketAcl",
            "Resource": "arn:aws:s3:::{}".format(cloudtrail_bucket)
        },
        {
            "Sid": "AWSCloudTrailWrite20150319",
            "Effect": "Allow",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
            "Condition": {
                "StringEquals": {
                    "s3:x-amz-acl": "bucket-owner-full-control"
                }
            }
        },
        {
            "Sid": "AWSCloudTrailHTTPSOnly20180329",
            "Effect": "Deny",
            "Principal": {
                "Service": "cloudtrail.amazonaws.com"
            },
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::{}/AWSLogs/{}/*".format(cloudtrail_bucket, account_id),
                "arn:aws:s3:::{}".format(cloudtrail_bucket)
            ],
            "Condition": {
                "Bool": {
                    "aws:SecureTransport": "false"
                }
            }
        }
    ]
}

print(policy)

In [ ]:
policy_json = json.dumps(policy)

In [ ]:
with open("policy.json", "w") as outfile: 
    json.dump(policy, outfile)

In [ ]:
!cat policy.json

In [ ]:
!aws s3api put-bucket-policy --bucket $cloudtrail_bucket --policy file://policy.json

# Create Cloud Trail

In [ ]:
cloudtrail = boto3.client('cloudtrail')
s3 = boto3.client('s3')

In [ ]:
trails = cloudtrail.describe_trails()

In [ ]:
print(trails)

In [ ]:
try:
    t = cloudtrail.create_trail(Name='dsoaws', S3BucketName=cloudtrail_bucket, IsMultiRegionTrail=True)
    trail_name = t['Name']
    trail_arn = t['TrailARN']
    cloudtrail.start_logging(Name=trail_arn)
    print("Cloud Trail created. Started logging.")
    print('--------------------------------------')
    print('New Trail name: {}'.format(trail_name))
    print('New Trail arn: {}'.format(trail_arn))
except ClientError as e:
    if e.response['Error']['Code'] == 'TrailAlreadyExistsException':
        print("Trail already exists. This is OK.")
        print('------------------')
        t = cloudtrail.get_trail(Name='dsoaws')
        trail_name = t['Trail']['Name']
        trail_arn = t['Trail']['TrailARN']
        print('Trail name: {}'.format(trail_name))
        print('Trail arn: {}'.format(trail_arn))
    else:
        print("Unexpected error: %s" % e)

## Get Default EventBridge EventBus

In [ ]:
events = boto3.client('events')

In [ ]:
response = events.describe_event_bus(Name='default')
eventbus_arn = response['Arn']
print('Bus {}'.format(eventbus_arn))

## Create Data Event Logging on CloudTrail for our S3 bucket

In [ ]:
!aws cloudtrail list-trails

In [ ]:
!aws cloudtrail get-event-selectors --trail-name $trail_name


In [ ]:
watched_bucket_arn = "arn:aws:s3:::{}/".format(watched_bucket)
print(watched_bucket_arn)

In [ ]:
event_selector = '\'[{ "ReadWriteType": "WriteOnly", "IncludeManagementEvents":true, "DataResources": [{ "Type": "AWS::S3::Object", "Values": ["' + watched_bucket_arn + '"] }] }]\''


In [ ]:
print(event_selector)

In [ ]:
!aws cloudtrail put-event-selectors --trail-name $trail_name --event-selectors $event_selector

## Create Custom EventBridge Rule

In [ ]:
pattern = {
  "source": [
    "aws.s3"
  ],
  "detail-type": [
    "AWS API Call via CloudTrail"
  ],
  "detail": {
    "eventSource": [
      "s3.amazonaws.com"
    ],
    "eventName": [
      "PutObject",
      "CompleteMultipartUpload",
      "CopyObject"
    ],
    "requestParameters": {
      "bucketName": [
        "{}".format(watched_bucket)
      ]
    }
  }
}

pattern_json = json.dumps(pattern)
print(pattern_json)

In [ ]:
response = events.put_rule(
    Name='S3-Trigger',
    EventPattern=pattern_json,
    State='ENABLED',
    Description='Triggers an event on S3 PUT',
    EventBusName='default'
)
print(response)

In [ ]:
rule_arn = response['RuleArn']
print(rule_arn)

# Add Target

## Create IAM Role

In [ ]:
iam = boto3.client('iam')

In [ ]:
iam_role_name_eventbridge = 'DSOAWS_EventBridge_Invoke_StepFunctions'

### Create AssumeRolePolicyDocument

In [ ]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "events.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [ ]:
try:
    iam_role_eventbridge = iam.create_role(
        RoleName=iam_role_name_eventbridge,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS EventBridge Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

### Get the Role ARN

In [ ]:
role_eventbridge = iam.get_role(RoleName=iam_role_name_eventbridge)
iam_role_eventbridge_arn = role_eventbridge['Role']['Arn']
print(iam_role_eventbridge_arn)

# Get the StepFunctions ARN and Name

In [ ]:
%store -r stepfunction_arn

In [ ]:
print(stepfunction_arn)

In [ ]:
%store -r stepfunction_name

In [ ]:
print(stepfunction_name)

# Define Eventbridge Policy

In [ ]:
eventbridge_sfn_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "states:StartExecution",
            "Resource": "*"
        }
    ]
}

print(eventbridge_sfn_policy)

# Create Policy Object

In [ ]:
try:
    policy_eventbridge_sfn = iam.create_policy(
      PolicyName='DSOAWS_EventBridgeInvokeStepFunction',
      PolicyDocument=json.dumps(eventbridge_sfn_policy)
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
        policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
        iam.create_policy_version(
            PolicyArn=policy_eventbridge_sfn_arn,
            PolicyDocument=json.dumps(eventbridge_sfn_policy),
            SetAsDefault=True)
        print("Policy updated.")
    else:
        print("Unexpected error: %s" % e)

# Get ARN

In [ ]:
policy_eventbridge_sfn_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_EventBridgeInvokeStepFunction'
print(policy_eventbridge_sfn_arn)

# Attach Policy To Role

In [ ]:
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_eventbridge_sfn_arn,
        RoleName=iam_role_name_eventbridge
    )
    print("Done.")
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Setup EventBridge Rule Target

In [ ]:
sfn = boto3.client('stepfunctions')

# Define Model Pipeline Inputs

In [ ]:
import time
timestamp = int(time.time())

execution_name = 'run-{}'.format(timestamp)
print(execution_name)

# Setup Input Data S3 Locations

In [ ]:
train_data_s3_uri = 's3://{}/bert-train/'.format(watched_bucket)

In [ ]:
%store -r max_seq_length

In [ ]:
print(max_seq_length)

In [ ]:
epochs=1
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=50
validation_steps=50
test_steps=50
train_instance_count=1
train_instance_type='ml.c5.9xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=True
enable_sagemaker_debugger=False
enable_checkpointing=False
enable_tensorboard=False
input_mode='File'
run_validation=False
run_test=False
run_sample_predictions=False

deploy_instance_count=1
deploy_instance_type='ml.m5.large'

Note:  Below, we are re-using the `sourcedir.tar.gz` (contains `tf_bert_reviews.py`) uploaded during a previous notebook's `sagemaker.estimator.TensorFlow.fit()` invocation.  We could manually copy the source to an S3 location and use this for the location of the `sourcedir.tar.gz`, but we choose to re-use for now.

In [ ]:
inputs = {
  "Training": {
    "AlgorithmSpecification": {
      "TrainingImage": "763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-training:2.1.0-cpu-py3".format(region),
      "TrainingInputMode": "{}".format(input_mode)
    },
    "OutputDataConfig": {
      "S3OutputPath": "s3://{}/training-pipeline-{}/models".format(bucket, execution_name)
    },
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 7200
    },
    "ResourceConfig": {
      "InstanceCount": train_instance_count,
      "InstanceType": "{}".format(train_instance_type),
      "VolumeSizeInGB": train_volume_size
    },
    "RoleArn": "{}".format(role),
    "InputDataConfig": [
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(train_data_s3_uri),              
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "train"
      },
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(train_data_s3_uri),
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "validation"
      },
      {
        "DataSource": {
          "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "{}".format(train_data_s3_uri),
            "S3DataDistributionType": "ShardedByS3Key"
          }
        },
        "ChannelName": "test"
      }
    ],
    "HyperParameters": {
      "epochs": "{}".format(epochs),
      "learning_rate": "{}".format(learning_rate),
      "epsilon": "{}".format(epsilon),
      "train_batch_size": "{}".format(train_batch_size),
      "validation_batch_size": "{}".format(validation_batch_size),
      "test_batch_size": "{}".format(test_batch_size),
      "train_steps_per_epoch": "{}".format(train_steps_per_epoch),
      "validation_steps": "{}".format(validation_steps),
      "test_steps": "{}".format(test_steps),
      "use_xla": "{}".format(str(use_xla).lower()),
      "use_amp": "{}".format(str(use_amp).lower()),
      "max_seq_length": "128".format(max_seq_length),
      "freeze_bert_layer": "{}".format(str(freeze_bert_layer).lower()),
      "enable_sagemaker_debugger": "{}".format(str(enable_sagemaker_debugger).lower()),
      "enable_checkpointing": "{}".format(str(enable_checkpointing).lower()),
      "enable_tensorboard": "{}".format(str(enable_tensorboard).lower()),
      "run_validation": "{}".format(str(run_validation).lower()),
      "run_test": "{}".format(str(run_test).lower()),
      "run_sample_predictions": "{}".format(str(run_sample_predictions).lower()),
      "sagemaker_submit_directory": "\"s3://{}/{}/estimator-source/source/sourcedir.tar.gz\"".format(bucket, stepfunction_name),
      "sagemaker_program": "\"tf_bert_reviews.py\"",
      "sagemaker_enable_cloudwatch_metrics": "false",
      "sagemaker_container_log_level": "20",
      "sagemaker_job_name": "\"training-pipeline-{}/estimator-source\"".format(execution_name),
      "sagemaker_region": "\"{}\"".format(region),
      "model_dir": "\"s3://{}/training-pipeline-{}/estimator-source/model\"".format(bucket, execution_name)
    },  
    "TrainingJobName": "estimator-training-pipeline-{}".format(execution_name),
    "DebugHookConfig": {
      "S3OutputPath": "s3://{}/".format(bucket)
    }
  },
  "Create Model": {
    "ModelName": "training-pipeline-{}".format(execution_name),
    "PrimaryContainer": {
      "Image": "763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference:2.1-cpu".format(region),
      "Environment": {
        "SAGEMAKER_PROGRAM": "null",
        "SAGEMAKER_SUBMIT_DIRECTORY": "null",
        "SAGEMAKER_ENABLE_CLOUDWATCH_METRICS": "false",
        "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
        "SAGEMAKER_REGION": "{}".format(region)
      },
      "ModelDataUrl": "s3://{}/training-pipeline-{}/models/estimator-training-pipeline-{}/output/model.tar.gz".format(bucket, execution_name, execution_name)
    },
    "ExecutionRoleArn": "{}".format(role)
  },
  "Configure Endpoint": {
    "EndpointConfigName": "training-pipeline-{}".format(execution_name),
    "ProductionVariants": [
      {
        "InitialInstanceCount": deploy_instance_count,
        "InstanceType": "{}".format(deploy_instance_type),
        "ModelName": "training-pipeline-{}".format(execution_name),
        "VariantName": "AllTraffic"
      }
    ]
  },
  "Deploy": {
    "EndpointConfigName": "training-pipeline-{}".format(execution_name),
    "EndpointName": "training-pipeline-{}".format(execution_name)
  }
}

In [ ]:
inputs_json = json.dumps(inputs)

print(inputs_json)

## Create EventBridge Rule Target

In [ ]:
# Check for exsting targets
targets = events.list_targets_by_rule(
    Rule='S3-Trigger',
    EventBusName='default'
)

In [ ]:
number_targets = len(targets['Targets'])

if number_targets > 0:
    for target in targets['Targets']:
        print(target['Id'])
        events.remove_targets(
            Rule='S3-Trigger',
            EventBusName='default',
            Ids=[target['Id']],
        Force=True
)
    print("Target: " +target['Id']+ " removed.")
else:
    print("No targets defined yet.")

In [ ]:
import uuid

target_id = str(uuid.uuid4())

response = events.put_targets(
    Rule='S3-Trigger',
    EventBusName='default',
    Targets=[
        {
            'Id': target_id,
            'Arn': stepfunction_arn,
            'RoleArn': iam_role_eventbridge_arn,
            'Input': inputs_json
        }
    ]
)

In [ ]:
print(response)

# Check Number of StepFunction Invocations **Before** the S3 Trigger

In [ ]:
execution_list_before_uploading = sfn.list_executions(stateMachineArn=stepfunction_arn)

number_of_executions_before_uploading = len(execution_list_before_uploading['executions'])

print(number_of_executions_before_uploading)

# Upload to S3 and Trigger a StepFunction Invocation

In [ ]:
print('Uploading training data to "{}" to trigger a new training pipeline.'.format(train_data_s3_uri))

In [ ]:
!aws s3 cp ./data-tfrecord/bert-train/part-algo-2-amazon_reviews_us_Digital_Software_v1_00.tfrecord $train_data_s3_uri

# Check Number of StepFunction Invocations **After** the S3 Trigger

In [ ]:
import time

time.sleep(30)

In [ ]:
execution_list_after_uploading = sfn.list_executions(stateMachineArn=stepfunction_arn)

print(execution_list_after_uploading)

In [ ]:
number_of_executions_after_uploading = len(execution_list_after_uploading['executions'])

print(number_of_executions_after_uploading)

In [ ]:
current_execution = execution_list_after_uploading['executions'][0]

current_execution_arn = current_execution['executionArn']

print(current_execution_arn)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/states/home?region={}#/executions/details/{}">Step Functions Pipeline</a></b>'.format(region, current_execution_arn)))


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();